# Graph Embedding for Recommendation System

Recommendation problems can be modeled on a graph e.g. a bipartite graphn of users and movies can be constructed where there is a link between a user *u* and a movie *m* if *u* has watched *m* and the weight of the link is equal to the rating that u gave for m. The graph abstraction also allows for adding more information e.g. directors of the movies as additional nodes in the graph and each movie has an edge to its director. 

Graphs provide a unified framework to study recommendation problems, in fact, the adjacency matrix of the user-movie bipartite graph is the matrix of the ratings that users have given for the movies. Matrix factorization of the adjacency matrix provides an embedding in a low-dimensional latent factor space for each node (users and movies) in the graph. 

   *Matrix factorization is an example of **graph embedding** i.e. a low-dimensional dense embedding in Euclidean space of every node the graph.*

**Recommendation** : Find a low-dimensional embedding of nodes (users, movies, directors etc.) such that the 'affinity/similarity' between nodes is mapped to the vector similarity in the embedding space. Further, the vector similarity provides a ranking function to order movies for a given user. 

With the popularity of neural networks, new methods of obtaining graph embeddings have been proposed, most notably [**DeepWalk**](https://arxiv.org/abs/1403.6652) and [**node2vec**](https://snap.stanford.edu/node2vec/) are two unsupervised techniques for graph embedding. Both these methods are inspired by word embedding models like **word2vec**. In this notebook, we will employ **node2vec** to obtain latent factors for users and movies and, same as for matrix factorization, use the vector similarity as ranking function. 

## Data

We will use the MovieLens data which contains ratings of movies by users. The data which is publically available from [MovieLens Website](https://grouplens.org/datasets/movielens/). We are using 100k data which has 100k ratings. 

In [1]:
import pandas as pd
import numpy as np

In [2]:
from scipy.sparse import coo_matrix, csr_matrix
from scipy.sparse.linalg import svds, norm
from scipy.spatial.distance import cosine
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
import operator
from collections import defaultdict

In [4]:
data_path = '../data/'

In [5]:
rating_df = pd.read_csv(data_path + 'ratings.csv', sep=',', header=0)

In [6]:
rating_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [7]:
rating_df.shape

(100836, 4)

In [8]:
rating_df.userId.nunique()

610

In [9]:
rating_df.movieId.nunique()

9724

- The data has over 100k ratings by 610 users on 9724 movies

In [10]:
max(rating_df.userId), max(rating_df.movieId)

(610, 193609)

  - The movie ids do not follow an order. 
  - 9724 movies have been selected that users with id 1 to 610 have rated so as to have 100k ratings. 

### Movies Information

We are also provided the titles and genres of the movies in a separate file. 

In [11]:
movie_df = pd.read_csv(data_path + 'movies.csv', sep=',', header=0)

In [12]:
movie_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [13]:
movie_df[movie_df['title'].str.contains('Avengers')]

,movieId,title,genres
1611,2153,"Avengers, The (1998)",Action|Adventure
6148,44020,Ultimate Avengers (2006),Action|Animation|Children|Sci-Fi
7693,89745,"Avengers, The (2012)",Action|Adventure|Sci-Fi|IMAX
8551,115727,Crippled Avengers (Can que) (Return of the 5 D...,Action|Adventure
8686,122892,Avengers: Age of Ultron (2015),Action|Adventure|Sci-Fi
8693,122912,Avengers: Infinity War - Part I (2018),Action|Adventure|Sci-Fi
9153,147657,Masked Avengers (1981),Action
9488,170297,Ultimate Avengers 2 (2006),Action|Animation|Sci-Fi


For the sake of completion and comparison, we start by studying matrix factorization based embeddings for recommendation. 

## Matrix Factorization 

A very popular technique for recommendation systems. We factorize the user-item matrix to obtain the user factors and item factors which are the low-dimensional embeddings such that 'similar' user/items are mapped to 'nearby' points. Moreover, the user and the movies are embedded to the same space, which provides a way to compute user-movie similarity. 

Create a matrix of ratings

In [14]:
ratings_mat = np.ndarray(
    shape=(np.max(rating_df.movieId.values), np.max(rating_df.userId.values)),
    dtype=np.uint8)
ratings_mat[rating_df.movieId.values-1, rating_df.userId.values-1] = rating_df.rating.values

In [15]:
ratings_mat.shape

(193609, 610)

Normalize the rating matrix

In [16]:
normalised_mat = ratings_mat - np.asarray([(np.mean(ratings_mat, 1))]).T

We will use **Singular Value Decomposition (SVD)** for factorizing the matrix. I'm delibately skipping any discussion of the mathematics of SVD. Since the user-movie rating matrix is very sparse, it is more efficient to use the implementation from scipy.sparse. 

The number of the latent-factors is chosen to be 50 i.e. top-50 singular values of the SVD are considered. The choice of the number of latent factors is a hyperparameter of the model, and requires a more sophisticated analysis to tune. We provide no reason for the choice of 50. 

In [17]:
n_factors = 50

In [18]:
A = normalised_mat.T / np.sqrt(ratings_mat.shape[0] - 1)
U, S, V = svds(A, n_factors)

In [19]:
U.shape

(610, 50)

In [20]:
V.shape

(50, 193609)

In [21]:
movie_factors = V.T
user_factors = U

Instead of representing each movie as a sparse vector of the ratings of all 360,000 possible users for it, after factorizing the matrix each movie will be represented by a 50 dimensional dense vector.

### Evaluation and understanding

Let's study some examples to have a qualitative understanding. Cosine similarity of the latent factors of two movies signifies how similar the movies are.

In [22]:
idx = 260
movie_df[movie_df.movieId == idx].title.values[0],  movie_df[movie_df.movieId == 1196].title.values[0]

('Star Wars: Episode IV - A New Hope (1977)',
 'Star Wars: Episode V - The Empire Strikes Back (1980)')

In [23]:
1.0 - cosine(movie_factors[259], movie_factors[1195])

0.8777832979913568

In [24]:
movie_df[movie_df.movieId == 1210].title.values[0], 1.0 - cosine(movie_factors[259], movie_factors[1209])

('Star Wars: Episode VI - Return of the Jedi (1983)', 0.8518636866885215)

In [25]:
movie_df[movie_df.movieId == 1].title.values[0], 1.0 - cosine(movie_factors[259], movie_factors[0])

('Toy Story (1995)', 0.20152844265131886)

The similarity of the 'Star Wars: Episode IV - A New Hope' is higher for the movies 'Star Wars: Episode V - The Empire Strikes Back' and 'Star Wars: Episode VI - Return of the Jedi' and is much lower for 'Toy Story'. Moreover, the 'Star Wars: Episode VI' is closer to 'Star Wars: Episode IV' than the 'Star Wars: Episode V'.  

Let's compute k-neareast neighboring movies for a given movie. 

In [26]:
def get_similar_movies_matrix_factorization(data, movieid, top_n=10):
    index = movieid - 1 # Movie id starts from 1
    movie = movie_df[movie_df.movieId == movieid].title.values[0]
    movie_row = data[index].reshape(1,-1)
    similarity = cosine_similarity(movie_row, data)
    sort_indexes = np.argsort(-similarity)[0]
    return {'movie': movie, 'sim_movies': [movie_df[movie_df.movieId == id].title.values[0] for id in sort_indexes[:top_n] + 1]}

In [27]:
movie_id = 260
get_similar_movies_matrix_factorization(movie_factors, movie_id)

{'movie': 'Star Wars: Episode IV - A New Hope (1977)',
 'sim_movies': ['Star Wars: Episode IV - A New Hope (1977)',
  'Star Wars: Episode V - The Empire Strikes Back (1980)',
  'Star Wars: Episode VI - Return of the Jedi (1983)',
  'Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)',
  'Lesson Faust (1994)',
  'Touch (1997)',
  'Inferno (2016)',
  'Beverly Hills Chihuahua (2008)',
  'Matrix, The (1999)',
  'Star Wars: Episode III - Revenge of the Sith (2005)']}

In [28]:
movie_id = 1
get_similar_movies_matrix_factorization(movie_factors, movie_id)

{'movie': 'Toy Story (1995)',
 'sim_movies': ['Toy Story (1995)',
  'Toy Story 2 (1999)',
  'Adventures of Pinocchio, The (1996)',
  'Eddie (1996)',
  'Children of the Corn IV: The Gathering (1996)',
  'Twister (1996)',
  'Sudden Death (1995)',
  'Dear God (1996)',
  'Kazaam (1996)',
  'Sunset Park (1996)']}

### Recommendation

Since the user and movies are in the same space, we can also compute movies similar to a user. A recommendation model can be defined as showing movies similar to the given user.  

In [32]:
def get_recommendations_matrix_factorization(userid, user_factors, movie_factors, top_n=10):
    user_vec = user_factors[userid - 1].reshape(1,-1)
    similarity = cosine_similarity(user_vec, movie_factors)
    sort_indexes = np.argsort(-similarity)[0]
    return [movie_df[movie_df.movieId == id].title.values[0] for id in sort_indexes[:top_n] + 1]   

In [33]:
top_recos = get_recommendations_matrix_factorization(1, user_factors, movie_factors)
top_recos

['Best Men (1997)',
 "Gulliver's Travels (1939)",
 'Newton Boys, The (1998)',
 'Teenage Mutant Ninja Turtles III (1993)',
 'Welcome to Woop-Woop (1997)',
 'Romancing the Stone (1984)',
 'Wolf Man, The (1941)',
 'Red Dawn (1984)',
 'Goonies, The (1985)',
 'Howard the Duck (1986)']

In [34]:
top_recos = get_recommendations_matrix_factorization(2, user_factors, movie_factors)
top_recos

['The Jinx: The Life and Deaths of Robert Durst (2015)',
 'Visit, The (2015)',
 'Shutter Island (2010)',
 'Iron Man & Hulk: Heroes United (2013)',
 "Hachiko: A Dog's Story (a.k.a. Hachi: A Dog's Tale) (2009)",
 'Girl with the Dragon Tattoo, The (2011)',
 'Inception (2010)',
 'The Hungover Games (2014)',
 'Django Unchained (2012)',
 'Inglourious Basterds (2009)']

## Graph Embeddings

Extending the same formulation of mapping users and movies to the same low-dimensional space, we discuss unsupervised graph embeddings.  

First, create a user-movie graph with edge weights as the ratings.  

In [35]:
import networkx as nx

In [37]:
user_item_edgelist = rating_df[['userId', 'movieId', 'rating']]
user_item_edgelist.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [38]:
user_item_edgelist.shape

(100836, 3)

Since userids and movieids both start from 1, and thus same id can correspond to a user and a movie. We will map the ids to unique integers. 

In [39]:
user2dict = dict()
movie2dict = dict()
cnt = 0
for x in user_item_edgelist.values:
    usr = (x[0], 'user')
    movie = (x[1], 'movie')
    if usr in user2dict:
        pass
    else:
        user2dict[usr] = cnt
        cnt += 1
    if movie in movie2dict:
        pass
    else:
        movie2dict[movie] = cnt
        cnt += 1

In [40]:
len(user2dict), len(movie2dict)

(610, 9724)

In [41]:
len(user2dict) + len(movie2dict)

10334

Define the user-movie graph.

In [42]:
user_movie_graph = nx.Graph()

In [43]:
for x in user_item_edgelist.values:
    usr = (x[0], 'user')
    movie = (x[1], 'movie')
    user_movie_graph.add_node(user2dict[usr])
    user_movie_graph.add_node(movie2dict[movie])
    user_movie_graph.add_edge(user2dict[usr], movie2dict[movie], weight=float(x[2]))

In [44]:
user_movie_graph.number_of_edges()

100836

In [45]:
user_movie_graph.number_of_nodes()

10334

### DeepWalk

DeepWalk embeds every node of a homogeneous graph into a low-dimensional space such that nodes that co-occur in same 'context' on truncated random walks are mapped to nearby points. We will not go into details of the model, for further details [read](https://arxiv.org/abs/1403.6652).  

DeepWalk works in following steps:
   - Generate fixed-length random walks starting from each node.
   - Create context nodes for each (center) node as those occuring within a fixed size window around the center node on a random walk. 
   - Skip-gram model to map center node to the context node. 

The implementation of **DeepWalk** is borrowed heavily from [node2vec repository](https://github.com/aditya-grover/node2vec) which is a bit different from original DeepWalk e.g. it uses *negative sampling* whereas the original DeepWalk paper used *hierarchical sampling* for the **skip-gram model**. 

To create embeddings from the context and non-context pairs, we are using Gensim python library. One can easily use Google **word2vec** or Facebook **fasttext** for this task. 

In [46]:
import node2vec 
from gensim.models import Word2Vec

In [47]:
G = node2vec.Graph(user_movie_graph, is_directed=False, p=1, q=1)

In [48]:
# Compute the transition probabilities based on the edge weights. 
G.preprocess_transition_probs()

Compute the random walks. 
  - 10 walks for every node.
  - Each walk of length 80. 

In [49]:
walks = G.simulate_walks(num_walks=10, walk_length=100)

Walk iteration:
1 / 10
2 / 10
3 / 10
4 / 10
5 / 10
6 / 10
7 / 10
8 / 10
9 / 10
10 / 10


In [50]:
len(walks)

103340

Learn Embeddings via Gensim, which creates context/non-context pairs and then Skip-gram. 

In [51]:
def learn_embeddings(walks):
    '''
    Learn embeddings by optimizing the Skipgram objective using SGD.
    Uses Gensim Word2Vec.
    '''
    walks = [list(map(str, walk)) for walk in walks]
    model = Word2Vec(walks, size=50, window=10, min_count=0, sg=1, workers=8, iter=1)
    return model.wv

In [52]:
node_embeddings = learn_embeddings(walks)

The output of gensim is a specific type of key-value pair with keys as the string-ed node ids and the values are numpy array of embeddings, each of shape (50,)

In [53]:
node_embeddings['0']

array([-1.25098601e-01,  6.01470351e-01,  2.64803529e-01, -2.50079751e-01,
        1.08021811e-01, -4.57026027e-02,  6.71414495e-01,  3.82575393e-01,
       -3.29733878e-01,  2.25284090e-03,  2.32380778e-01,  8.54553804e-02,
        7.19717741e-02, -2.19303206e-01,  5.27384575e-04,  6.83498904e-02,
        8.37790728e-01, -1.65708214e-01,  5.46279773e-02, -7.83528909e-02,
        1.35477856e-01,  1.21959478e-01,  1.41724125e-01, -1.25758946e-01,
        2.74086416e-01, -3.70948762e-01, -6.41676128e-01,  6.02644086e-01,
       -4.97822076e-01,  6.00377358e-02,  2.69516826e-01, -4.41659153e-01,
       -2.13203043e-01,  4.30300862e-01,  2.91034758e-01,  1.88774347e-01,
        6.35144830e-01,  6.97985068e-02, -7.35652894e-02, -2.00328991e-01,
        4.03339565e-02, -2.56082833e-01,  1.81893006e-01,  4.24633592e-01,
       -1.73566341e-01,  6.46947265e-01, -1.22240573e-01,  1.84876751e-02,
        1.03271715e-01,  1.78193837e-01], dtype=float32)

In [54]:
node_embeddings['0'].shape

(50,)

## Evaluation

Let's look at the same examples that we used to qualitatively investigate the matrix factorization based embeddings.
- 260 = Star Wars IV
- 1196 = Star Wars V
- 1210 = Star Wars VI
- 1 = Toy Stroy

In [55]:
movie1 = str(movie2dict[(260, 'movie')])
movie2 = str(movie2dict[(1196, 'movie')])
1.0 - cosine(node_embeddings[movie1], node_embeddings[movie2])

0.8959806561470032

In [56]:
movie3 = str(movie2dict[(1210, 'movie')])
1.0 - cosine(node_embeddings[movie1], node_embeddings[movie3])

0.8984009623527527

In [57]:
movie4 = str(movie2dict[(1, 'movie')])
1.0 - cosine(node_embeddings[movie1], node_embeddings[movie4])

0.7448261380195618

Since we worked with integer ids for nodes, let's create reverse mapping dictionaries that map integer user/movie to their actual ids. 

In [58]:
reverse_movie2dict = {k:v for v,k in movie2dict.items()}
reverse_user2dict = {k:v for v,k in user2dict.items()}

In [59]:
node_vecs = [node_embeddings[str(i)] for i in range(cnt)]
node_vecs = np.array(node_vecs)
node_vecs.shape

(10334, 50)

Similar Movies

In [60]:
def get_similar_movies_graph_embeddings(movieid, movie_embed, top_n=10):
    movie_idx = movie2dict[movieid]
    query = movie_embed[movie_idx].reshape(1,-1)
    ranking = cosine_similarity(query, movie_embed)
    top_ids = np.argsort(-ranking)[0]
    top_movie_ids = [reverse_movie2dict[j] for j in top_ids if j in reverse_movie2dict][:top_n]
    sim_movies = [movie_df[movie_df.movieId == id[0]].title.values[0] for id in top_movie_ids]
    return sim_movies

In [61]:
get_similar_movies_graph_embeddings((260, 'movie'), node_vecs)[:10]

['Star Wars: Episode IV - A New Hope (1977)',
 'Star Wars: Episode VI - Return of the Jedi (1983)',
 'Star Wars: Episode V - The Empire Strikes Back (1980)',
 'Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)',
 'Independence Day (a.k.a. ID4) (1996)',
 'Matrix, The (1999)',
 'Spy Next Door, The (2010)',
 'Back to the Future (1985)',
 'Behind Enemy Lines II: Axis of Evil (2006)',
 'Mission: Impossible (1996)']

In [62]:
get_similar_movies_graph_embeddings((122912, 'movie'), node_vecs)[:10]

['Avengers: Infinity War - Part I (2018)',
 'Thor: Ragnarok (2017)',
 'Deadpool 2 (2018)',
 'Guardians of the Galaxy 2 (2017)',
 'Untitled Spider-Man Reboot (2017)',
 'Star Wars: The Last Jedi (2017)',
 'Mission: Impossible - Fallout (2018)',
 'Captain America: Civil War (2016)',
 'Justice League (2017)',
 'Incredibles 2 (2018)']

In [63]:
get_similar_movies_graph_embeddings((1, 'movie'), node_vecs)[:10]

['Toy Story (1995)',
 'Twister (1996)',
 'Independence Day (a.k.a. ID4) (1996)',
 'Lion King, The (1994)',
 'Mission: Impossible (1996)',
 'Twelve Monkeys (a.k.a. 12 Monkeys) (1995)',
 'Sunset Park (1996)',
 'Willy Wonka & the Chocolate Factory (1971)',
 'Jurassic Park (1993)',
 'Father of the Bride Part II (1995)']

### Recommendation

In [64]:
def get_recommended_movies_graph_embeddings(userid, movie_vecs, top_n=10):
    user_idx = user2dict[userid]
    query = movie_vecs[user_idx].reshape(1,-1)
    ranking = cosine_similarity(query, movie_vecs)
    top_ids = np.argsort(-ranking)[0]
    top_movie_ids = [reverse_movie2dict[j] for j in top_ids if j in reverse_movie2dict][:top_n]
    reco_movies = [movie_df[movie_df.movieId == id[0]].title.values[0] for id in top_movie_ids]
    return reco_movies

In [65]:
get_recommended_movies_graph_embeddings((1, 'user'), node_vecs, top_n=10)

['Best Men (1997)',
 'Newton Boys, The (1998)',
 "Gulliver's Travels (1939)",
 'Shaft (1971)',
 'Howard the Duck (1986)',
 'American Tail, An (1986)',
 'Song of the South (1946)',
 'Welcome to Woop-Woop (1997)',
 'Quiet Man, The (1952)',
 'Teenage Mutant Ninja Turtles III (1993)']

In [103]:
get_recommended_movies_graph_embeddings((2, 'user'), node_vecs, top_n=10)

['The Jinx: The Life and Deaths of Robert Durst (2015)',
 'Shutter Island (2010)',
 'The Drop (2014)',
 'Warrior (2011)',
 'Django Unchained (2012)',
 'Dark Knight Rises, The (2012)',
 'Inside Job (2010)',
 'Wolf of Wall Street, The (2013)',
 'The Great Hypnotist (2014)',
 'Ex Machina (2015)']

Compare recommendations generated by graph embeddings to those of matrix factorization. 

In [104]:
recos = set(get_recommended_movies_graph_embeddings((1, 'user'), node_vecs, top_n=10))
high_rated_movies = set([movie_df[movie_df.movieId == id].title.values[0] for id in rating_df[(rating_df['userId'] == 1) & (rating_df['rating'] >= 4.5)].movieId.values])
recos.intersection(high_rated_movies)

{'American Tail, An (1986)',
 "Gulliver's Travels (1939)",
 'Newton Boys, The (1998)',
 'Quiet Man, The (1952)',
 'Shaft (1971)'}

In [106]:
precision_at_10 = len(recos.intersection(high_rated_movies)) / 10
precision_at_10

0.5

In [107]:
recos_mf = set(get_recommendations_matrix_factorization(1, user_factors, movie_factors))
recos_mf.intersection(high_rated_movies)

{'Goonies, The (1985)',
 "Gulliver's Travels (1939)",
 'Newton Boys, The (1998)',
 'Red Dawn (1984)',
 'Wolf Man, The (1941)'}

In [109]:
precision_at_10_mf = len(recos_mf.intersection(high_rated_movies)) / 10
precision_at_10_mf

0.5

In [114]:
idx = 2
recos_ge = set(get_recommended_movies_graph_embeddings((idx, 'user'), node_vecs, top_n=10))
high_rated_movies = set([movie_df[movie_df.movieId == id].title.values[0] for id in rating_df[(rating_df['userId'] == idx) & (rating_df['rating'] >= 4.5)].movieId.values])
true_pos_ge = recos_ge.intersection(high_rated_movies)
print('precision at 10 for ge: ', len(true_pos_ge)/10)

recos_mf = set(get_recommendations_matrix_factorization(idx, user_factors, movie_factors))
true_pos_mf = recos_mf.intersection(high_rated_movies)
print('precision at 10 for mf: ', len(true_pos_mf)/10)

precision at 10 for ge:  0.4
precision at 10 for mf:  0.2


In [115]:
idx = 3
recos_ge = set(get_recommended_movies_graph_embeddings((idx, 'user'), node_vecs, top_n=10))
high_rated_movies = set([movie_df[movie_df.movieId == id].title.values[0] for id in rating_df[(rating_df['userId'] == idx) & (rating_df['rating'] >= 4.5)].movieId.values])
true_pos_ge = recos_ge.intersection(high_rated_movies)
print('precision at 10 for ge: ', len(true_pos_ge)/10)

recos_mf = set(get_recommendations_matrix_factorization(idx, user_factors, movie_factors))
true_pos_mf = recos_mf.intersection(high_rated_movies)
print('precision at 10 for mf: ', len(true_pos_mf)/10)

precision at 10 for ge:  1.0
precision at 10 for mf:  0.4


In [116]:
idx = 4
recos_ge = set(get_recommended_movies_graph_embeddings((idx, 'user'), node_vecs, top_n=10))
high_rated_movies = set([movie_df[movie_df.movieId == id].title.values[0] for id in rating_df[(rating_df['userId'] == idx) & (rating_df['rating'] >= 4.5)].movieId.values])
true_pos_ge = recos_ge.intersection(high_rated_movies)
print('precision at 10 for ge: ', len(true_pos_ge)/10)

recos_mf = set(get_recommendations_matrix_factorization(idx, user_factors, movie_factors))
true_pos_mf = recos_mf.intersection(high_rated_movies)
print('precision at 10 for mf: ', len(true_pos_mf)/10)

precision at 10 for ge:  0.2
precision at 10 for mf:  0.5


## Enriched network with additional information : Genres

Graphs make it easier to incorporate into the same framework additional information e.g. directors/actors of the movies. Since DeepWalk uses a neural network which can be trained in batches via stochaistic gradient descent, incorporating extra nodes, e.g actors to model a 3-step relation user-movie-actor, does not add any memory issues. As compared to tensor (matrix_ factorization. 

In the MovieLens data, we are also provided genres of the movies. We will add genres as nodes in our graph with edges to movies that belong to them. This provides an additional path for a random walk to connect two user (U-M-G-M-U), thus capturing more intricacies of the graph. 

Furthermore, this setting maps users,movies and genres to same space, which allows the computation of a user's affinity to a genre. User-genre relation can be exploited to handle **cold-start problem** i.e. to recommend a newly added movie to users. 

In [73]:
movie_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


#### Genres of the movies can be used as additional signal for better recommendations

In [74]:
movie_genre_edgelist = movie_df[['movieId', 'genres']]
movie_genre_edgelist.head()

,movieId,genres
0,1,Adventure|Animation|Children|Comedy|Fantasy
1,2,Adventure|Children|Fantasy
2,3,Comedy|Romance
3,4,Comedy|Drama|Romance
4,5,Comedy


In [75]:
cnt

10334

Map genres to indices. 

In [77]:
genre2int = dict()
for x in movie_genre_edgelist.values:
    genres = x[1].split('|')
    for genre in genres:
        if genre in genre2int:
            pass
        else:
            genre2int[genre] = cnt
            cnt += 1

In [78]:
cnt

10374

In [79]:
genre2int

{'Adventure': 10354,
 'Animation': 10355,
 'Children': 10356,
 'Comedy': 10357,
 'Fantasy': 10358,
 'Romance': 10359,
 'Drama': 10360,
 'Action': 10361,
 'Crime': 10362,
 'Thriller': 10363,
 'Horror': 10364,
 'Mystery': 10365,
 'Sci-Fi': 10366,
 'War': 10367,
 'Musical': 10368,
 'Documentary': 10369,
 'IMAX': 10370,
 'Western': 10371,
 'Film-Noir': 10372,
 '(no genres listed)': 10373}

Create the movie-genre graph.

In [80]:
movie_genre_graph = nx.Graph()
for x in movie_genre_edgelist.values:
    movie = (x[0], 'movie')
    genres = x[1].split('|')
    if movie in movie2dict:
        for genre in genres:
            if genre in genre2int:
                movie_genre_graph.add_node(movie2dict[movie])
                movie_genre_graph.add_node(genre2int[genre])
                movie_genre_graph.add_edge(movie2dict[movie], genre2int[genre], weight=1.0)
            else:
                pass

In [81]:
movie_genre_graph.number_of_nodes()

9744

In [83]:
rating_df.movieId.nunique()

9724

In [85]:
movie_genre_graph.number_of_edges()

22046

#### Combine the user-movie and movie-genre graph

In [88]:
user_movie_genre_graph =  nx.Graph()
user_movie_genre_graph.add_weighted_edges_from([(x,y,user_movie_graph[x][y]['weight']) for x,y in user_movie_graph.edges()])
user_movie_genre_graph.add_weighted_edges_from([(x,y,movie_genre_graph[x][y]['weight']) for x,y in movie_genre_graph.edges()])

In [89]:
user_movie_genre_graph.number_of_edges()

122882

In [90]:
list(user_movie_genre_graph.edges())[0]

(0, 1)

In [97]:
user_movie_genre_graph.number_of_nodes()

10354

### DeepWalk on Enriched Graph

The user-movie-genre graph is heterogeneous and random walk methods might not be appropriate. The random walks at movie nodes will be biased towards user nodes as compared to the genre nodes since there are way more user nodes connected to a movie node than the genre nodes (<=5). But for this discussion we will ignore the heterogeniety and treat all nodes as the same type. 

In [91]:
G_enriched = node2vec.Graph(user_movie_genre_graph, is_directed=False, p=1, q=1)

In [92]:
G_enriched.preprocess_transition_probs()

In [93]:
walks_enriched = G_enriched.simulate_walks(num_walks=10, walk_length=80)

Walk iteration:
1 / 10
2 / 10
3 / 10
4 / 10
5 / 10
6 / 10
7 / 10
8 / 10
9 / 10
10 / 10


In [94]:
node_embeddings_enriched = learn_embeddings(walks_enriched)

In [96]:
cnt

10374

In [98]:
node_vecs_enriched = [node_embeddings_enriched[str(i)] for i in range(cnt) if str(i) in node_embeddings_enriched]
node_vecs_enriched = np.array(node_vecs_enriched)
node_vecs_enriched.shape

(10354, 50)

## Evaluation enriched graph embeddings

### Similar Movies

In [99]:
get_similar_movies_graph_embeddings((260, 'movie'), node_vecs_enriched)[:10]

['Star Wars: Episode IV - A New Hope (1977)',
 'Star Wars: Episode V - The Empire Strikes Back (1980)',
 'Star Wars: Episode VI - Return of the Jedi (1983)',
 'Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)',
 'Matrix, The (1999)',
 'Groundhog Day (1993)',
 'Indiana Jones and the Last Crusade (1989)',
 'Lord of the Rings: The Fellowship of the Ring, The (2001)',
 'Gladiator (2000)',
 'Back to the Future (1985)']

In [100]:
get_similar_movies_graph_embeddings((260, 'movie'), node_vecs)[:10]

['Star Wars: Episode IV - A New Hope (1977)',
 'Star Wars: Episode VI - Return of the Jedi (1983)',
 'Star Wars: Episode V - The Empire Strikes Back (1980)',
 'Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)',
 'Independence Day (a.k.a. ID4) (1996)',
 'Matrix, The (1999)',
 'Spy Next Door, The (2010)',
 'Back to the Future (1985)',
 'Behind Enemy Lines II: Axis of Evil (2006)',
 'Mission: Impossible (1996)']

### Recommendation

In [101]:
get_recommended_movies_graph_embeddings((1, 'user'), node_vecs_enriched, top_n=10)

['Newton Boys, The (1998)',
 'Quiet Man, The (1952)',
 'Best Men (1997)',
 'Welcome to Woop-Woop (1997)',
 "Gulliver's Travels (1939)",
 'Rescuers, The (1977)',
 'Red Dawn (1984)',
 'American Tail, An (1986)',
 'Shaft (1971)',
 'Song of the South (1946)']

In [102]:
get_recommended_movies_graph_embeddings((1, 'user'), node_vecs_enriched, top_n=10)

['Newton Boys, The (1998)',
 'Quiet Man, The (1952)',
 'Best Men (1997)',
 'Welcome to Woop-Woop (1997)',
 "Gulliver's Travels (1939)",
 'Rescuers, The (1977)',
 'Red Dawn (1984)',
 'American Tail, An (1986)',
 'Shaft (1971)',
 'Song of the South (1946)']

In [125]:
idx = 6
recos_ge_enriched = set(get_recommended_movies_graph_embeddings((idx, 'user'), node_vecs_enriched, top_n=10))
high_rated_movies = set([movie_df[movie_df.movieId == id].title.values[0] for id in rating_df[(rating_df['userId'] == idx) & (rating_df['rating'] >= 4.5)].movieId.values])
true_pos_ge_enriched = recos_ge_enriched.intersection(high_rated_movies)
print('precision at 10 for ge_enriched: ', len(true_pos_ge)/10)

recos_ge = set(get_recommended_movies_graph_embeddings((idx, 'user'), node_vecs, top_n=10))
high_rated_movies = set([movie_df[movie_df.movieId == id].title.values[0] for id in rating_df[(rating_df['userId'] == idx) & (rating_df['rating'] >= 4.5)].movieId.values])
true_pos_ge = recos_ge.intersection(high_rated_movies)
print('precision at 10 for ge: ', len(true_pos_ge)/10)

recos_mf = set(get_recommendations_matrix_factorization(idx, user_factors, movie_factors))
true_pos_mf = recos_mf.intersection(high_rated_movies)
print('precision at 10 for mf: ', len(true_pos_mf)/10)

precision at 10 for ge_enriched:  0.5
precision at 10 for ge:  0.0
precision at 10 for mf:  0.0


To conclude : 
-  This discussion is devoid of any mathematical details, see [DeepWalk](https://arxiv.org/abs/1403.6652), [node2vec](https://snap.stanford.edu/node2vec/), [blog](https://towardsdatascience.com/a-gentle-introduction-to-graph-neural-network-basics-deepwalk-and-graphsage-db5d540d50b3).  
- The goal was to study recommendation problem via graph embeddings. No claims about this being the best model, although graph embedding (e.g. GCN) based models have seen great success in recommendation systems. 
- The evaluation as presented in sloppy - only focussing on movie-movie similarity which was studied qualitatively, and some calculations of precision@10. Ideally, we would split data into train and test - train for modeling and test for evaluation. 
- To quantitively evaluate the whole system, we would compute mean average precision at 10. 
- I hope to add more details in future. 